In [110]:
import requests
import bs4
from bs4 import BeautifulSoup as bs
import urllib.request as ureq
import csv
import re

In [111]:
Australiaforum_url = 'https://www.australiaforum.com/visas-immigration/'
Ausforum = requests.get(Australiaforum_url)
#print(Ausforum)
soup = bs(Ausforum.text, 'html.parser')
#print(soup.prettify())
#soup.find_all('a')
#tabs = [link['href'] for link in soup.find_all('a', {'class':"n_first_a"})
#print(tabs)
#tab_titles = list(map(lambda h: h.text.strip(), tabs))
#print(tab_titles)
#for links in tabs:
    #print links.get("href")
#THIS PRINTS ALL HREF LINKS
#for link in soup.find_all('a'):
    #print(link.get('href'))

#THIS PRINTS THE TOP TABS WITH CLASS: N_FIRST_A:
#for link in soup.find_all("a", "n_first_a", href=True):
 #   print(link['href'])
    


In [112]:
title = soup.find("a", {'title': (re.compile('Last Page'))}).get('href')

In [113]:
last_num = int(re.search(r"index(\d+)\.", title).groups()[0])
print(last_num)

1859


In [109]:
for i in range(1, last_num):
    print('https://www.australiaforum.com/visas-immigration/index'+str(i)+'.html')

https://www.australiaforum.com/visas-immigration/index1.html
https://www.australiaforum.com/visas-immigration/index2.html
https://www.australiaforum.com/visas-immigration/index3.html
https://www.australiaforum.com/visas-immigration/index4.html
https://www.australiaforum.com/visas-immigration/index5.html
https://www.australiaforum.com/visas-immigration/index6.html
https://www.australiaforum.com/visas-immigration/index7.html
https://www.australiaforum.com/visas-immigration/index8.html
https://www.australiaforum.com/visas-immigration/index9.html
https://www.australiaforum.com/visas-immigration/index10.html
https://www.australiaforum.com/visas-immigration/index11.html
https://www.australiaforum.com/visas-immigration/index12.html
https://www.australiaforum.com/visas-immigration/index13.html
https://www.australiaforum.com/visas-immigration/index14.html
https://www.australiaforum.com/visas-immigration/index15.html
https://www.australiaforum.com/visas-immigration/index16.html
https://www.austr

In [18]:
#soup.find_all('a')
#topics = soup.find_all('a', {'id':'thread_title_250194'})
#print(topics)
#all_topics = list(map(lambda a: a.text.strip(), topics))
#print(all_topics)
#titles = list(map(lambda h: h.text.strip(), topics))
#print(titles)
#print (soup.prettify())

titles = []

for link in soup.find_all("a", {'id': (re.compile('thread_title_'))}):
    href = link.get('href')
    titles.append(href)              
#print(titles[0:])

#links = titles.find_all("href")
#print(links)
#Ausforum = requests.get(Australiaforum_url)
#soup2 = bs(title_url.text, 'html.parser')
#print(soup2)

#titles = soup.find_all("a", {'id': (re.compile('thread_title_'))})
#print(titles)
#topics = list(map(lambda h: h.text.strip(), titles))
#print(topics)
#url = requests.get(title_links["href"])
#print(url)



In [38]:
def extract_content(i):
    #This code pulls out data from a single link at a time
    murl = requests.get(titles[i])
    #print(murl)
    msoup = bs(murl.text, 'html.parser')
    #print(msoup)
    messages = msoup.find_all('div', {'id': (re.compile('post_message_'))})
    #print(messages)
    content = list(map(lambda h: h.text.strip(), messages))
    return (content)

In [39]:
all_page_contents = []

for i in range(1, len(titles)):
    content = extract_content(i)
    all_page_contents.append(content)
    print(content)

['Hi All\n\nOn this forum we are reguarly asked about recommending a registered migration agent. There are 4 on this forum that post quite regularly that we recommend. I have listed them below in alphabetical order:\n\n------\nCCMS\nNick van Voorst\nMARN 0640648\nWebsite: http://www.coralcoastmigrationservice.com.au\n\n------\nJeremy Hooper\nJeremy W Hooper \nMARN 0107902\nWebsite: http://4australianvisas.com\n\n------\nMarkNortham\nMark Northam\nMARN 1175508\nWebsite: http://nlaw.com.au \n\n----\nwrussell\nWestly Russell\nMARN 0316072\nWebsite: Pinoy Australia Pty Ltd\n\n----\npaulhand\nPaul Hand\nMARN 1801974\nSunCoast Migration Ltd\n\n-------\n\nFor more information about registered migration agents including how to find one, how to become one, the code of conduct, etc. you can visit the OMARA website:  https://www.mara.gov.au/']
["Hi everyone, \r\nI noticed that there is a waiting room for people waiting on the Onshore Partner visa, but nothing similar (that I could find!) for thos

['sad that these immigration sites nowadays seem to be mainly concerned with attracting business for migration agents . Most questions can be answered by referring to government site and by googling. If your situation is UNIQUE , surely a migration agent could not answer your question definitivelty and would just give an opinion based on logic . I saw the following from an agent and I doubt it is true if one could check the official statistics . It is the overselling of their services which I do not like with even the simplest question on one immigration forum, NOT this one , getting standard reply " consult an agent " . Obviously they are constantly looking for business as anyone would be in their position I guess . perhaps a modest fee for answering a question would be good for those visa applicants and their sponsors who can not be bothered to do own research or too busy to do so . anyway , best of luck . \n\nPartner visas are easy,’ some visa applicants and their sponsors say, dism

["Hello\n\r\nI am a PhD Student with scholarship in Griffith university, currently living in Australia. I have lodged for my husbands 500 subsequent entrant dependent visa on the 5th of June. It has been 5 weeks but haven't heard anything yet.\n\r\nWhat is the normal processing time for this visa\r\nHow much time until case officer gets assigned\r\nHow much time till we get hap id for medical \n\r\nAny help would be really appreciated", 'It can be up to 3 months, sometimes longer. I hope you addressed the GTE in detail. I have seen several cases similar to yours where the subsequent entrant has been refused on GTE grounds. If that happens to you let me know. I have been able to get visas for subsequent entrants from high-risk countries where the original applicant was refused.', 'Quote:\n\n\n\n\r\n\t\t\t\t\tOriginally Posted by Jeremy Hooper\n\n\nIt can be up to 3 months, sometimes longer. I hope you addressed the GTE in detail. I have seen several cases similar to yours where the subs

["Hi all,\n\r\nI have applied on my partner's 485 visa as subsequent entrant from offshore through a lawyer on the 02/09/2016 and yet to receive an acknowledge letter.\n\r\nMy application was decision ready (lawyer advised that medical will have to be done upon DIBP request)\n\r\nPlease advise current processing time for 485 subsequent entrant visa. Her 485 will finish on 29th December, 2016 and we do not have much time.\n\r\nThanks in advance.", 'Hi,\nDid you get any information about your application. I lodged my application on 2/9/2016. And I got acknowledge letter today. My agent said that it will take about 3 months to get the decision. \nHow about you?', 'Hi CD277,\n\r\nSorry, I missed your post.\n\r\nI have also received my acknowledgement letter within a week of lodging the application.\n\r\nMy lawyer advised that an outcome of the application might take 3-4 months.\n\r\nDo you know if there is any way to track progress of the application?', "Quote:\n\n\n\n\r\n\t\t\t\t\tOrigina

['I\'m a Kiwi and my husband is a UK citizen. We\'ve been married almost 3 years and we\'ve submitted our application for the 461 Family Visa here in London. \n\nSince submitting the application, "global processing times" has gone from 7 months to 12 months so I\'m curious to know if anyone in the UK has recently been granted their visa and how long it took?\n\nThe application was submitted 2 weeks ago and the money has been taken from our account - but that\'s the only \'progress\' we know of...\n\nIt\'s so hard waiting to know so any relevant insight on London processing times is totally appreciated', 'Hi\r\nAny update on your progress? About to lodge our application. \r\nThanks!', 'My Husband (UK/Irish applied for a 461 Visa 1st Jan 2018 in London. We are still waiting with no word! The only way I know that it was reviled it that payment was taken a few days later. Managed to get through on the phone and they said the processing time is still as what is relflextes online but since t

["Hi everyone!\n\r\nMy partner and I are coming up to the 2nd Stage of our 820/801 Partner Visa application. We are currently in the process of gathering up all of our evidence so we are prepared when the 2 year mark rolls around in the next 2 months. I have a potentially stupid question regarding Stat Decs.\n\r\nI believe we both have to fill out Statutory Declarations about our relationship, however the Department of Home Affairs website isn't really clear on this since they updated the department and the website. I was only able to find this on the website (immi.homeaffairs.gov.au/form-listing/forms/stat_dec_sponsor.pdf)\n\r\nDoes anyone know if this is just for the sponsor to fill out, or if it can be used for both the sponsor and the applicant ? From memory, I think there used to be a different PDF available for the applicant, but I can't find it anywhere on their official website which is what is confusing me. We applied in 2017 and it looks like this may have changed a bit since

['Hello again, \n\r\nI am still waiting for my Partner visa 309 since August 2018. \r\nI have some questions. \n\r\n1. Can I apply for sponsored tourist visa (12months) while waiting for the PMV 309?  What are the pros and cons? \n\r\n2. Can I apply for student visa while waiting for the PMV 309? What are the pros and cons... \n\r\nI hope someone can help us. Thanks in advance.', 'Quote:\n\n\n\n\r\n\t\t\t\t\tOriginally Posted by joie\n\n\n1. Can I apply for sponsored tourist visa (12months) while waiting for the PMV 309?  What are the pros and cons? \n\r\nI hope someone can help us. Thanks in advance.\n\n\n\nWhile we waited for the PMV, we applied for a 12 month Visitor Visa (600) but not the sponsored one.\n\r\nWorking is obviously not permitted on the 600, but at least you can be together.\n\r\nSome get granted the full 12 months, others 12 months but must leave every 3 months, some get only 3 months and others get refused.\n\r\nIt often depends on what country you apply in.\n\r\nSo,

In [37]:
print(len(all_page_contents))

0


In [19]:
#This code pulls out data from a single link at a time
murl = requests.get(titles[7])
msoup = bs(murl.text, 'html.parser')
#print(msoup)
messages = msoup.find_all('div', {'id': (re.compile('post_message_'))})
#print(messages)
content = list(map(lambda h: h.text.strip(), messages))
print (content)

['Hey everyone,\n\n just wondering if there is anyone applying from the Nairobi office. 2019 new year had me very excited and then I came back to this site and I now kinda feel like I have to adjust my expectations. \n\nWe applied September 5 2018 for a 309 partner visa. I have a 2 year old with my husband who is Australian. She has citizenship n lives with me. My immiaccount currently shows "further assessment" from 2018 16th Nov. I havent heard from anyone and I know I am in a high risk country.\nHave provided medical and police records as had been requested.\n\nQuestions. Anyone here applying from the same office and got their 309 without too much hassle time wise and anything else? (Take that with a pinch of salt)\n\nAnyone know exactly why the character federal police certificate is needed from Aussie sponsors and what exactly they need from it for a positive visa outcome. (Hope that makes sense)', 'I also applied through Nairobi for my PMV in June 2018. Medical and police checks 

In [15]:
for link in soup.find_all("a", "smallfont", href=True):
    print(link['href'])

https://www.australiaforum.com/visas-immigration/index2.html
https://www.australiaforum.com/visas-immigration/index3.html
https://www.australiaforum.com/visas-immigration/index4.html
https://www.australiaforum.com/visas-immigration/index5.html
https://www.australiaforum.com/visas-immigration/index6.html
https://www.australiaforum.com/visas-immigration/index7.html
https://www.australiaforum.com/visas-immigration/index8.html
https://www.australiaforum.com/visas-immigration/index9.html
https://www.australiaforum.com/visas-immigration/index10.html
https://www.australiaforum.com/visas-immigration/index11.html
https://www.australiaforum.com/visas-immigration/index51.html
https://www.australiaforum.com/visas-immigration/index101.html
https://www.australiaforum.com/visas-immigration/index501.html
https://www.australiaforum.com/visas-immigration/index1001.html
https://www.australiaforum.com/visas-immigration/index2.html
https://www.australiaforum.com/visas-immigration/index1859.html
https://www

In [16]:
#TESTING WOTH JUST ONE URL TO SEE IF THE TAGS AND REGEX WORKS
#url = 'https://www.australiaforum.com/visas-immigration/231066-461-visa-timeline-any-updates-2017-a.html'

#testforum = requests.get(url)
#testsoup = bs(testforum.text, 'html.parser')
#messages = testsoup.find_all('div', {'id': (re.compile('post_message'))})
#print(messages)
#content = list(map(lambda h: h.text.strip(), messages))
#print(content)

In [17]:
for i in requests.get(titles[0:]):
    bs(murl.text, 'html.parser')
    messages = msoup.find_all('div', {'id': (re.compile('post_message_'))})
    content = list(map(lambda h: h.text.strip(), messages))

InvalidSchema: No connection adapters were found for '['https://www.australiaforum.com/visas-immigration/250194-permanent-partner-visas-801-100-eligibility-notification.html', 'https://www.australiaforum.com/visas-immigration/252474-registered-migration-agents.html', 'https://www.australiaforum.com/visas-immigration/80537-offshore-partner-visa-309-100-waiting-room.html', 'https://www.australiaforum.com/visas-immigration/286589-australian-citizenship-2019-onward-applications.html', 'https://www.australiaforum.com/visas-immigration/288715-sad-these-immigration-sites-concerned-attracting-business.html', 'https://www.australiaforum.com/visas-immigration/212762-permanent-partner-visa-subclass-801-waiting-group.html', 'https://www.australiaforum.com/visas-immigration/229034-500-subsequent-entrant-dependent-visa-processing-time.html', 'https://www.australiaforum.com/visas-immigration/284215-309-visa-application-nairobi-kenya.html', 'https://www.australiaforum.com/visas-immigration/282769-aust